## Sprint 2: Data

The dataset that I have chosen covers traffic accidents in the United States from 2016-2023. The data was collected from various government sources, such as state and federal departments of transportation, and compiled into one dataset. I chose this dataset because of its size and robust feature set, which should allow for many interesting visualizations and analysis.

The current version of the datset is from Kaggle: https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents?resource=download

Below are citations for the papers for the original creation of the dataset:

Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, and Rajiv Ramnath. “A Countrywide Traffic Accident Dataset.”, 2019.

Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, Radu Teodorescu, and Rajiv Ramnath. "Accident Risk Prediction based on Heterogeneous Sparse Data: New Dataset and Insights." In proceedings of the 27th ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, ACM, 2019.

### Data Cleaning

In [ ]:
# import dependencies
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import addfips
import datetime
import time

In [ ]:
# read in data from csv
df_raw = pd.read_csv('US_Accidents_March23.csv')
# display preview of dataset
df_raw.head()

In [3]:
df = df_raw

In [ ]:
# get basic info about dataset
df.info()

In [ ]:
df.isna().sum()

Variables will be removed if they don't seem to provide any opportunity for interesting analysis, or if they seem redundant because there are already other, better variables than can be used for a certain aspect of analysis.

In [4]:
# remove unwanted or redundant variables
df = df.drop(columns={'ID', 'Source', 'End_Lat', 'End_Lng', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Wind_Direction', 'Weather_Timestamp', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Turning_Loop'})

In [5]:
# remove entries where some more important variables are null
df.dropna(subset={'Description', 'Street', 'City', 'Sunrise_Sunset', 'Temperature(F)','Weather_Condition', 'Temperature(F)', 'Visibility(mi)' }, inplace=True)

For the remaining variables, wind speed and precipitation, the null values will be filled with 0

In [6]:
df = df.fillna(0.0)

The dataset now has no null values. A few more steps are needed to prepare the dataset for use with the dashboard. The first of these is to add FIPS codes for counties and states, which will be used to make it easier to plot the data on a choropleth.

In [ ]:
af = addfips.AddFIPS()

df['StateFIPS'] = '0'
df['CountyFIPS'] = '0'

for index, row in df.iterrows():
    if index % 10000 == 0:
        print(index)
    county = row['County']
    state = row['State']
    state_fips = af.get_state_fips(state)
    county_fips = af.get_county_fips(county, state=state)
    df.at[index, 'StateFIPS'] = state_fips
    df.at[index, 'CountyFIPS'] = county_fips

The next step is to add unix timestamps to each row, which will make it easier to filter data points by date.

In [ ]:
def get_unix_timestamp(data_row):
    date = pd.to_datetime(data_row['Start_Time'])
    unix_timestamp  = int(time.mktime(date.timetuple()))
    return unix_timestamp

df['timestamp'] = df.apply(get_unix_timestamp, axis=1)

Next, a few more columns can be dropped.

In [ ]:
df = df.drop(columns={'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'Description'})

Finally, the dataset needs to be truncated for use with the dashboard, since the full dataset is larger than the RAM provided by Render. A sample of 400k data points will be used.

In [ ]:
df = df.sample(400000)

### Data Analysis

In [ ]:
df.to_csv('clean_data.csv', index=False)

### Potential UI and Visualizations

Two good UI elements would be a drop down allowing selection of multiple numeric variables (like date), as well as sliders for selected numerical variables to select the range to be visualized. Another good UI element could be radio selection for categorical variables to visualize, like weather condition. One more could be checkboxes to select entries with certain boolean variables true, like whether a certain traffic feature was present.

The best visualization for this dataset is probably an interactive map. The map can be broken down into either states or counties, and it can be colored according to the frequency of accidents based on the filters selected by the UI elements above. Another possible visualization is a scatter plot of two differect selectable variables to see how they correlate. Another possible visualization is a pie chart showing what proportion of all total accidents have a certain trait, such as breakdown of accident frequency by time of day.